<a href="https://colab.research.google.com/github/sauravsingla/Dream-11/blob/main/Dream_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Optimising Dream 11 Team Selection ##

### Section 1.1 Data Preparation :
1) Prepare a Score Vector (The mean of a Player's Score across 10 Matches is considered) <br>
2) Prepare Standard Deviation Vector( to capture variability or inconsistencies in player's scores). This is to be used in Mean Variance Optimisation

In [ ]:
import pandas as pd
import numpy as np
from gurobipy import *

cric_df=pd.read_csv('E:\MSBA\Players_Score.csv')
cric_df.head()

,Player Name,Player Type,Team,Retrospective Scores_10,Retrospective Scores_1,Retrospective Scores_2,Retrospective Scores_3,Retrospective Scores_4,Retrospective Scores_5,Retrospective Scores_6,Retrospective Scores_7,Retrospective Scores_8,Retrospective Scores_9,Price
0,V Kohli,BAT,INDIA,278,374,387,372,380,477,481,357,474,477,11.5
1,S Dhawan,BAT,INDIA,299,337,275,147,121,72,226,141,125,547,9.5
2,J Denly,BAT,ENGLAND,208,222,226,242,263,259,276,258,256,272,9.0
3,R Sharma,BAT,INDIA,454,251,442,436,133,437,442,250,294,461,11.0
4,J Root,BAT,ENGLAND,424,228,378,21,417,365,429,425,420,119,9.0


#### Find mean and Standard Deviation of the scores of all 30 Players ###

In [ ]:
score_df=cric_df.iloc[0:,3:cric_df.shape[1]-1] # Separate the score data from other details

In [ ]:
sd_score=[np.std(score_df.iloc[i,:].values) for i in range(score_df.shape[0])] #Standard Deviation to measure inconsistency
scores=[np.mean(score_df.iloc[i,:].values) for i in range(score_df.shape[0])] #Score of each player

In [ ]:
#########Other Parameters Set-up############

# Intialise a Player Type(Batsman,Bowler,Allrounder or Wicket Keeper) Matrix
ptype=np.zeros((4,30))

# Initialize a Team Matrix (The Dream 11 Team has to be formed from two competing Teams)
team=np.zeros((2,30))

# Fill in the Player and Team matrices with values from Dataset
for i in range(cric_df.shape[0]):
    if(cric_df.loc[i,'Player Type']=='BAT'):
        ptype[0,i]=1
    elif(cric_df.loc[i,'Player Type']=='BOWL'):
        ptype[1,i]=1
    elif(cric_df.loc[i,'Player Type']=='ALL'):
        ptype[2,i]=1
    else:
        ptype[3,i]=1
    if(cric_df.loc[i,'Team']=='INDIA'):
        team[0,i]=1
    else:
        team[1,i]=1

# Prepare a Budget Vector
price=cric_df['Price'].values

# Prepare the lower bound and upper bound for Player Type . This is to be read as :
# The no. of batsman can be between 3 and 6 (lb[0] and ub[0] respectively)
# Similarly, the no. of bowlers can be between 3 and 6; No. of allrounders can be between 1 and 4; 
# and No of wicketkeeper can be between 1 and 4
lb=[3,3,1,1]
ub=[6,6,4,4]

# There can only be 1 Captain, 1 Vice Captain and the rest 9 players will be considered as normal players
player_role=[9,1,1]


N=cric_df.shape[0] # No of Players
M=len(player_role) #No of Player Roles
K=ptype.shape[0] #No. of Player Types

# Risk_aversion captures the risk tolerance of a Dream 11 Player. 
#From a problem perspective, we aim to punish a player for his inconsistency. Risk_aversion captures the degree of punishment

risk_aversion=0

### Section 1.2 Stating the Objective Function and Constraints. Displaying the Output ###

In [ ]:
def dream11_optimisation():
    cric = Model("cricket")

    # Creat variables
    x=cric.addVars(M, N, vtype=GRB.BINARY, name = "x")


    # Set objective : A Vice Captain has his score multiplied by 1.5 and A captain has his score multiplied by 2.
    # Additionally, a Player is punished for his inconsistency (A form of Mean Variance Optimisation, 
    # used mostly in financial portfolio management)
    cric.setObjective( quicksum(scores[j]*x[0,j] for j in range(N))\
                  +quicksum(1.5*scores[j]*x[1,j] for j in range(N))\
                  +quicksum(2*scores[j]*x[2,j] for j in range(N))\
                  -risk_aversion*(quicksum(x[i,j]*sd_score[j] for i in range(M) for j in range(N))), GRB.MAXIMIZE)
    
    # The Player can either not be selected and if selected, can only take in one of the role : Either a Captain, Vice Captain or
    # an ordinary player
    cric.addConstrs(( quicksum(x[i,j] for i in range(M)) <= 1 for j in range(N) ))
    
    # The below constraint ensures that there are 9 ordinary players, only 1 Vice Captain and only 1 Captain in the Team
    cric.addConstrs(quicksum(x[i,j] for j in range(N)) ==player_role[i] for i in range(M))
    
    # Only 100 Points are available for Spending
    cric.addConstr(quicksum(price[j]*x[i,j] for i in range(M) for j in range(N)) <=100)
    
    # The no of Batsman,Bowler, Allrounder and WK has to be within the specified ranges
    cric.addConstrs(quicksum(x[i,j]*ptype[k,j] for i in range(M) for j in range(N))<=ub[k] for k in range(K))

    cric.addConstrs(quicksum(x[i,j]*ptype[k,j] for i in range(M) for j in range(N))>=lb[k] for k in range(K))

    # There can be a maximum of 7 players from one team
    cric.addConstrs(quicksum(x[i,j]*team[k,j] for i in range(M) for j in range(N))<=7 for k in range(team.shape[0]))

    return cric,x

In [ ]:
# risk_aversion can vary between 1 to 10: 
# A risk_aversion score of 0 means the risk taking apetite is huge. This Dream 11 Player doesn't take inconsistencies in 
# cricket players performance into account while selecting his Playing 11. 
# 10 signifies that the risk-taking apetite of the Dream 11 player is almost negligible. He/She would prefer only consistent
# players in his/her playing 11
while (True):
    risk_aversion = int(input("Enter your risk tolerance : "))
    if(risk_aversion>10 or risk_aversion < 0 ):
        print("Invalid Input!!. Please Choose a tolerance limit between 0 and 10"+"(both inclusive)")
        continue
    else:
        print ('Risk Tolerance of the Dream-11 Player is',risk_aversion)
        # setup the model again
        dr_cric,x = dream11_optimisation()
        
        #Supressing the optimization output
        dr_cric.setParam('OutputFlag', False )
        
        # Solving the model
        dr_cric.optimize()
        
        # Display the Output
        print('The Optimum Team for the upcoming match based on your risk tolerance is shown below:- \n')
        for v in dr_cric.getVars():
            if(int(v.VarName[2])==2 and v.x > 0):
                index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
                print('The Captain of the Team is',cric_df.loc[index,'Player Name'] + \
                  '('+ cric_df.loc[index,'Player Type'] +')'+ ' from '+ cric_df.loc[index,'Team'])
        for v in dr_cric.getVars():
            if(int(v.VarName[2])==1 and v.x > 0):
                index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
                print('The Vice Captain of the Team is',cric_df.loc[index,'Player Name'] + \
                      '('+ cric_df.loc[index,'Player Type'] +')'+ ' from '+ cric_df.loc[index,'Team']+'\n')
        print('Other Players:- \n')
        for v in dr_cric.getVars():
            if(int(v.VarName[2])==0 and v.x > 0):
                index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
                print(cric_df.loc[index,'Player Name'] +'('+ cric_df.loc[index,'Player Type'] +')'+ \
                      ' from '+ cric_df.loc[index,'Team'])
        print('The expected score of the match is ',dr_cric.objVal)
        # Ask from User if he want's to select a team on other risk tolerance levels, apart from the one entered earlier
        choice = input("Enter (Y/N) if you wan't to see team combination for other risk_tolerance levels : \n")
        if(choice.upper()=='Y' ):
            continue
        else:
            print('No more Team Selections')
            break

Enter your risk tolerance : 2
Risk Tolerance of the Dream-11 Player is 2
The Optimum Team for the upcoming match based on your risk tolerance is shown below:- 

The Captain of the Team is V Kohli(BAT) from INDIA
The Vice Captain of the Team is KL Rahul(BAT) from INDIA

Other Players:- 

J Denly(BAT) from ENGLAND
J Roy(BAT) from ENGLAND
M Ali(ALL) from ENGLAND
J Bumrah(BOWL) from INDIA
B Kumar(BOWL) from INDIA
C Woakes(BOWL) from ENGLAND
J Archer(BOWL) from ENGLAND
U Yadav(BOWL) from INDIA
J Buttler(WK) from ENGLAND
The expected score of the match is  3175.1654335748462
Enter (Y/N) if you wan't to see team combination for other risk_tolerance levels : 
N
No more Team Selections


## Section 2: Sensitivity Analysis ##

#### Sometimes, the pitch on which the match will be played on, can be curated to become more batsman friendly. The team management would like to include more batsman in the team.In such cases, the lower bound for the number of batsman, can be increased. ###

In [ ]:
# Change Parameters
lb[0]=6 #Change the minimum no of batsman to be included in the team to 6
ub[3]=1 #A Wicket-Keeper is also a batsman, So we need to constraint the no.of Wicket-Keeper to 1, if we are selecting 
# 6 specialist batsmen.

risk_aversion=0 # Reset Risk_aversion =0

# setup the model again
dr_cric,x = dream11_optimisation()

# Solving the model
dr_cric.optimize()

Optimize a model with 44 rows, 90 columns and 540 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+02, 8e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 6 rows and 3 columns
Presolve time: 0.00s
Presolved: 38 rows, 87 columns, 456 nonzeros
Variable types: 0 continuous, 87 integer (87 binary)
Found heuristic solution: objective 4082.0500000

Root relaxation: objective 4.125675e+03, 44 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 4125.67500    0    2 4082.05000 4125.67500  1.07%     -    0s
H    0     0                    4120.9500000 4125.67500  0.11%     -    0s
     0     0 infeasible    0      4120.95000 4120.95000  0.00%     -    0s

Explored 1 nodes (44 simplex iterations) in 0.15 seconds
Thread count was 

In [ ]:
print('The Optimum Team for the upcoming match is shown below:- \n')
for v in dr_cric.getVars():
    if(int(v.VarName[2])==2 and v.x > 0):
        index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
        print('The Captain of the Team is',cric_df.loc[index,'Player Name'] +'('+ cric_df.loc[index,'Player Type'] +')'+ ' from '+ cric_df.loc[index,'Team'])
for v in dr_cric.getVars():
    if(int(v.VarName[2])==1 and v.x > 0):
        index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
        print('The Vice Captain of the Team is',cric_df.loc[index,'Player Name'] +'('+ cric_df.loc[index,'Player Type'] +')'+ ' from '+ cric_df.loc[index,'Team']+'\n')
print('Other Players:- \n')
for v in dr_cric.getVars():
    if(int(v.VarName[2])==0 and v.x > 0):
        index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
        print(cric_df.loc[index,'Player Name'] +'('+ cric_df.loc[index,'Player Type'] +')'+ ' from '+ cric_df.loc[index,'Team'])

The Optimum Team for the upcoming match is shown below:- 

The Captain of the Team is V Kohli(BAT) from INDIA
The Vice Captain of the Team is KL Rahul(BAT) from INDIA

Other Players:- 

J Denly(BAT) from ENGLAND
R Sharma(BAT) from INDIA
J Root(BAT) from ENGLAND
J Roy(BAT) from ENGLAND
L Plunkett(BOWL) from ENGLAND
J Archer(BOWL) from ENGLAND
U Yadav(BOWL) from INDIA
J Buttler(WK) from ENGLAND
K Jadav(ALL) from INDIA


#### The team management has decided to rest V Kohli, who has been the choice for 'Captain'  for both risk-taking and risk-averse Dream 11 participants. Who shall be the new Captain ?###

In [ ]:
# Change Parameters
#Revert the changes made in the previous Scenario
lb[0]=3 
ub[3]=4
risk_aversion=0

# setup the model again
dr_cric,y = dream11_optimisation()

# Add an additional constraint to make sure that V Kohli is not selected in Playing 11
index_cap=cric_df.index[cric_df['Player Name']=='V Kohli'].values[0]
dr_cric.addConstr (quicksum(y[i,index_cap] for i in range(M))==0)

# Solving the model
dr_cric.optimize()

Optimize a model with 45 rows, 90 columns and 543 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+02, 8e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 2789.8000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.01s
Presolved: 43 rows, 87 columns, 513 nonzeros
Variable types: 0 continuous, 87 integer (87 binary)

Root relaxation: objective 3.964900e+03, 54 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3964.9000000 3964.90000  0.00%     -    0s

Explored 0 nodes (54 simplex iterations) in 0.06 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 3964.9 2789.8 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.964900000000e+03, best b

In [ ]:
print('The Optimum Team for the upcoming match is shown below:- \n')
for v in dr_cric.getVars():
    if(int(v.VarName[2])==2 and v.x > 0):
        index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
        print('The Captain of the Team is',cric_df.loc[index,'Player Name'] +'('+ cric_df.loc[index,'Player Type'] +')'+ ' from '+ cric_df.loc[index,'Team'])
for v in dr_cric.getVars():
    if(int(v.VarName[2])==1 and v.x > 0):
        index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
        print('The Vice Captain of the Team is',cric_df.loc[index,'Player Name'] +'('+ cric_df.loc[index,'Player Type'] +')'+ ' from '+ cric_df.loc[index,'Team']+'\n')
print('Other Players:- \n')
for v in dr_cric.getVars():
    if(int(v.VarName[2])==0 and v.x > 0):
        index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
        print(cric_df.loc[index,'Player Name'] +'('+ cric_df.loc[index,'Player Type'] +')'+ ' from '+ cric_df.loc[index,'Team'])

The Optimum Team for the upcoming match is shown below:- 

The Captain of the Team is KL Rahul(BAT) from INDIA
The Vice Captain of the Team is R Sharma(BAT) from INDIA

Other Players:- 

J Root(BAT) from ENGLAND
J Roy(BAT) from ENGLAND
J Bumrah(BOWL) from INDIA
L Plunkett(BOWL) from ENGLAND
J Archer(BOWL) from ENGLAND
U Yadav(BOWL) from INDIA
D Karthik(WK) from INDIA
J Buttler(WK) from ENGLAND
K Jadav(ALL) from INDIA


#### At times depending on weather conditions, the team which is batting second, is at an disadvantage. The Dew on the pitch ensures that the ball experiences unpredictable swings. In this case, the bowling team should include more bowlers in the team ###

In [ ]:
# Change Parameters
lb[1]=5 #The minimum no of bowlers included is 5
risk_aversion=0

# setup the model again
dr_cric,y = dream11_optimisation()

# Solving the model
dr_cric.optimize()

Optimize a model with 44 rows, 90 columns and 540 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+02, 8e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 3090.0500000
Presolve time: 0.00s
Presolved: 44 rows, 90 columns, 531 nonzeros
Variable types: 0 continuous, 90 integer (90 binary)

Root relaxation: objective 4.127350e+03, 58 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    4127.3500000 4127.35000  0.00%     -    0s

Explored 0 nodes (58 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 4127.35 3090.05 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.127350000000e+03, best bound 4.127350000000e+03, gap 0.0000%

In [ ]:
print('The Optimum Team for the upcoming match is shown below:- \n')
for v in dr_cric.getVars():
    if(int(v.VarName[2])==2 and v.x > 0):
        index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
        print('The Captain of the Team is',cric_df.loc[index,'Player Name'] +'('+ cric_df.loc[index,'Player Type'] +')'+ ' from '+ cric_df.loc[index,'Team'])
for v in dr_cric.getVars():
    if(int(v.VarName[2])==1 and v.x > 0):
        index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
        print('The Vice Captain of the Team is',cric_df.loc[index,'Player Name'] +'('+ cric_df.loc[index,'Player Type'] +')'+ ' from '+ cric_df.loc[index,'Team']+'\n')
print('Other Players:- \n')
for v in dr_cric.getVars():
    if(int(v.VarName[2])==0 and v.x > 0):
        index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
        print(cric_df.loc[index,'Player Name'] +'('+ cric_df.loc[index,'Player Type'] +')'+ ' from '+ cric_df.loc[index,'Team'])

The Optimum Team for the upcoming match is shown below:- 

The Captain of the Team is V Kohli(BAT) from INDIA
The Vice Captain of the Team is KL Rahul(BAT) from INDIA

Other Players:- 

R Sharma(BAT) from INDIA
J Root(BAT) from ENGLAND
J Bumrah(BOWL) from INDIA
L Plunkett(BOWL) from ENGLAND
B Kumar(BOWL) from INDIA
J Archer(BOWL) from ENGLAND
U Yadav(BOWL) from INDIA
J Buttler(WK) from ENGLAND
K Jadav(ALL) from INDIA


### Randomly generating a new match's scores of each players

In [ ]:
new_score=[]

for i in range(len(scores)):
    new_score.append(max(np.random.normal(scores[i],sd_score[i],1)[0],0))

In [ ]:
new_score

[401.1411266580253,
 30.990636780869636,
 257.59937423759396,
 561.8942433806479,
 348.08585367845006,
 384.5010810678119,
 263.212931249031,
 161.6264076162899,
 206.63641895871666,
 177.4267027934267,
 160.56230236754297,
 196.20265217061356,
 93.58810890118214,
 244.77840885074707,
 292.043335067028,
 464.23480323653735,
 247.6121235673849,
 261.00947784693756,
 216.340729041747,
 220.74080700685676,
 272.4998694986309,
 278.4161893149656,
 353.2014756937042,
 229.66468220089976,
 226.21259534891513,
 377.94221204591884,
 200.32284514026531,
 363.7772067584925,
 168.99931678955525,
 206.61073321311082]

In [ ]:
s=0
for v in dr_cric.getVars():
    index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
    if(int(v.VarName[2])==2 and v.x > 0):
        s+=new_score[index]*2
    elif(int(v.VarName[2])==1 and v.x > 0):
        s+=new_score[index]*1.5
    elif(int(v.VarName[2])==0 and v.x > 0):
        s+=new_score[index]
        
print(s)        

3909.3396018576154


In [ ]:
for risk_aversion in range(11):        
        print ('Risk Tolerance of the Dream-11 Player is',risk_aversion)
        # setup the model again
        dr_cric,x = dream11_optimisation()
        
        #Supressing the optimization output
        dr_cric.setParam('OutputFlag', False )
        
        # Solving the model
        dr_cric.optimize()
        
        # Display the Output
        print('The Optimum Team for the upcoming match based on your risk tolerance is shown below:- \n')
        for v in dr_cric.getVars():
            if(int(v.VarName[2])==2 and v.x > 0):
                index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
                print('The Captain of the Team is',cric_df.loc[index,'Player Name'] + \
                  '('+ cric_df.loc[index,'Player Type'] +')'+ ' from '+ cric_df.loc[index,'Team'])
        for v in dr_cric.getVars():
            if(int(v.VarName[2])==1 and v.x > 0):
                index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
                print('The Vice Captain of the Team is',cric_df.loc[index,'Player Name'] + \
                      '('+ cric_df.loc[index,'Player Type'] +')'+ ' from '+ cric_df.loc[index,'Team']+'\n')
        print('Other Players:- \n')
        for v in dr_cric.getVars():
            if(int(v.VarName[2])==0 and v.x > 0):
                index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
                print(cric_df.loc[index,'Player Name'] +'('+ cric_df.loc[index,'Player Type'] +')'+ \
                      ' from '+ cric_df.loc[index,'Team'])
        s1=0
        for v in dr_cric.getVars():
            index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
            if(int(v.VarName[2])==2 and v.x > 0):
                s1+=scores[index]*2
            elif(int(v.VarName[2])==1 and v.x > 0):
                s1+=scores[index]*1.5
            elif(int(v.VarName[2])==0 and v.x > 0):
                s1+=scores[index]
        print('Expected Score: ',s1)
        
        s=0
        for v in dr_cric.getVars():
            index=int(v.VarName[v.VarName.index(',')+1:v.VarName.index(']')])
            if(int(v.VarName[2])==2 and v.x > 0):
                s+=new_score[index]*2
            elif(int(v.VarName[2])==1 and v.x > 0):
                s+=new_score[index]*1.5
            elif(int(v.VarName[2])==0 and v.x > 0):
                s+=new_score[index]
        
        print('Actual Score: ',s)  
        

Risk Tolerance of the Dream-11 Player is 0
The Optimum Team for the upcoming match based on your risk tolerance is shown below:- 

The Captain of the Team is V Kohli(BAT) from INDIA
The Vice Captain of the Team is KL Rahul(BAT) from INDIA

Other Players:- 

R Sharma(BAT) from INDIA
J Root(BAT) from ENGLAND
J Bumrah(BOWL) from INDIA
L Plunkett(BOWL) from ENGLAND
B Kumar(BOWL) from INDIA
J Archer(BOWL) from ENGLAND
U Yadav(BOWL) from INDIA
J Buttler(WK) from ENGLAND
K Jadav(ALL) from INDIA
Expected Score:  4127.349999999999
Actual Score:  4408.546915290469
Risk Tolerance of the Dream-11 Player is 1
The Optimum Team for the upcoming match based on your risk tolerance is shown below:- 

The Captain of the Team is V Kohli(BAT) from INDIA
The Vice Captain of the Team is KL Rahul(BAT) from INDIA

Other Players:- 

J Denly(BAT) from ENGLAND
J Roy(BAT) from ENGLAND
M Ali(ALL) from ENGLAND
J Bumrah(BOWL) from INDIA
L Plunkett(BOWL) from ENGLAND
B Kumar(BOWL) from INDIA
J Archer(BOWL) from ENGLAN